In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
dataset=pd.read_csv("dataset.csv")

In [13]:
print(dataset.shape)
data_size=len(dataset)
print (data_size)

(16602, 3)
16602


In [14]:
import re
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

corpus=[]

for i in range(0,data_size):
    tweet = re.sub('[^a-zA-Z]', ' ', dataset['text'][i])
    tweet=tweet.lower()
    tweet=tweet.split()
    ps=PorterStemmer()
    tweet=[ps.stem(word) for word in tweet if word not in set(stopwords.words('english')) and len(word)>2]
    tweet=' '.join(tweet)
    corpus.append(tweet)
    
    

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


In [5]:
corpus_temp=corpus

In [22]:
corpus=corpus_temp

In [23]:
Y=dataset.iloc[:,2].values
Y=list(Y)
print (len(corpus))
seen = set()
result = []
removed_indices=[]
for i in range(0,data_size):
    if corpus[i] not in seen:
        seen.add(corpus[i])
        result.append(corpus[i])
    else:
        removed_indices.insert(0,i)
corpus=result
for i in removed_indices:
    del(Y[i])
print (len(corpus))
print (len(Y))

16602
14613
14613


In [24]:
corpus=corpus[0:5000]
Y=Y[0:5000]
print(len(corpus))
print(len(Y))

5000
5000


In [25]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=6000)
X=cv.fit_transform(corpus).toarray()

In [26]:
print (X.shape)

(5000, 6000)


In [27]:
from sklearn.cross_validation import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.20,random_state=0)


In [28]:
print (len(X_train))
print (len(Y_train))
print (len(X_test))
print (len(Y_test))


4000
4000
1000
1000


In [29]:
# Fitting Naive Bayes to the Training set
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, Y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, y_pred)


In [30]:
print (cm)

[[789  92  47]
 [ 11   2   0]
 [ 30   1  28]]


In [32]:
from sklearn.metrics import accuracy_score
print (accuracy_score(Y_test,y_pred))

0.819
